## Header Transform, Verification and Alignment of NIfTI Images

This set of scripts was developed to ensure that the headers of brain MRI and mask images are consistent, and to correct them when discrepancies are found. This is critical in medical imaging workflows—especially for segmentation or post-processing—because mismatched header information (such as affine matrices or metadata) can lead to misaligned overlays, incorrect interpretations, or processing errors.

### Overview of the Workflow:

1. **Script 1**: Creates a dummy binary mask from a loaded MRI image using SimpleITK. It assigns the same header and spatial information as the MRI image to the mask.
2. **Script 2**: Creates a dummy zero-filled mask using `nibabel`, saving it with the affine matrix of the original image.
3. **Script 3**: Compares the headers of two NIfTI images and prints their differences, helping identify mismatches.
4. **Script 4**: Creates a binary mask by setting voxels to 255 where the source image has non-zero values. It applies the header and affine of a second image.
5. **Script 5**: Loads an image and forces it to adopt the affine and header of a reference image, saving a corrected output.

These tools are essential for ensuring spatial consistency across datasets and are particularly useful in preprocessing for tasks like training deep learning models, applying filters, or anatomical alignment.

In [ ]:
!pip install SimpleITK


In [ ]:
import SimpleITK as sitk

# Load the MRI image
mri_image = sitk.ReadImage('AAA417_62_hm_t1_space_sag_p2_0.7iso_nativ_reg2ref_Warped.nii')

# Convert the image to a NumPy array
mri_array = sitk.GetArrayFromImage(mri_image)

# Create a mask where intensity values are greater than zero
dummy_mask_array = (mri_array > 0).astype(int)

# Convert the NumPy array back to a SimpleITK image
dummy_mask = sitk.GetImageFromArray(dummy_mask_array)

# Copy metadata from the original MRI image to the mask
dummy_mask.CopyInformation(mri_image)

# Save the mask
sitk.WriteImage(dummy_mask, 'dummy_mask.nii')

In [ ]:
import nibabel as nib
import numpy as np

def create_dummy_mask(input_img_path, output_mask_path):
    # Load the input image
    input_img = nib.load(input_img_path)
    input_data = input_img.get_fdata()

    # Create a zero-filled mask with the same shape
    dummy_mask = np.zeros_like(input_data)

    # Save the dummy mask with the same affine
    nib.save(nib.Nifti1Image(dummy_mask, input_img.affine), output_mask_path)

# Paths
input_img_path = "AAA417_62_hm_t1_space_sag_p2_0.7iso_nativ_reg2ref_Warped.nii"
output_mask_path = "dummy_mask.nii"

# Generate mask
create_dummy_mask(input_img_path, output_mask_path)
print("✅ Dummy mask created successfully.")

In [ ]:
import nibabel as nib

def compare_headers(original_img_path, mask_img_path):
    # Load both images
    original_img = nib.load(original_img_path)
    mask_img = nib.load(mask_img_path)

    # Extract headers
    original_header = original_img.header
    mask_header = mask_img.header

    # Compare headers
    if original_header == mask_header:
        print("✅ Headers of original image and mask are identical.")
    else:
        print("❌ Headers differ between the original image and the mask.")
        print("Original header:\n", original_header)
        print("Mask header:\n", mask_header)

# Paths
original_img_path = "ZHL066_56_hm_tse_ac-pc_512_p2_PVS_reg2ref_Warped (1).nii.gz"
mask_img_path = "ZHL066_BG_new.nii.gz"

# Run comparison
compare_headers(original_img_path, mask_img_path)

Los encabezados de la imagen original y la máscara son iguales.


In [ ]:
import nibabel as nib
import numpy as np

def create_mask(image_1, image_2):
    # Get data arrays
    data_1 = image_1.get_fdata()
    data_2 = image_2.get_fdata()

    # Create binary mask: 255 where image_1 is non-zero
    mask = np.where((data_1 != 0), 255, 0)

    # Create NIfTI image using affine and header from image_2
    mask_image = nib.Nifti1Image(mask, affine=image_2.affine, header=image_2.header)

    return mask_image

# Paths
path_image_1 = "p0AFH292_62_hm_t1_space_sag_p2_0.7iso_nativ.nii.gz"
path_image_2 = "AFH292_56_hm_tse_ac-pc_512_p2_reg2ref_Warped.nii.gz"

# Load images
image_1 = nib.load(path_image_1)
image_2 = nib.load(path_image_2)

# Generate mask
mask_image = create_mask(image_1, image_2)

# Save result
mask_path = "mask_AFH292.nii.gz"
nib.save(mask_image, mask_path)
print("✅ Mask created and saved at:", mask_path)

Máscara creada y guardada en: mascara_AFH292.nii.gz


In [ ]:
import nibabel as nib

def load_image_with_reference_header(reference_img_path, img_to_align_path):
    # Load reference image
    reference_img = nib.load(reference_img_path)
    reference_header = reference_img.header.copy()
    reference_affine = reference_img.affine

    # Load image to align
    img_to_align = nib.load(img_to_align_path)

    # Create aligned image with reference header and affine
    aligned_img = nib.Nifti1Image(img_to_align.get_fdata(), reference_affine, header=reference_header)

    return aligned_img

# Paths
reference_path = "ZHL066_56_hm_tse_ac-pc_512_p2_PVS_reg2ref_Warped (1).nii.gz"
img_to_align_path = "ZHL066_BG (2).nii.gz"

# Load and align image
aligned_image = load_image_with_reference_header(reference_path, img_to_align_path)

# Save aligned image
aligned_output_path = "ZHL066_BG_new.nii.gz"
nib.save(aligned_image, aligned_output_path)
print(f"✅ Aligned image saved at: {aligned_output_path}")

Se ha guardado la imagen alineada con el mismo encabezado que la imagen de referencia en: ZHL066_BG_new.nii.gz
